# Description
In this example I evolve an optimizer on a single logistic regression dataset with batch size of 1. I set number of steps to 100 because otherwise it is painfully slow (even though it's as fast as it can be). Realistically one should at least increase number of steps to 1000 to 10,000, or even better, evaluate on multiple datasets. But this might work too.

In [ ]:
from myai.torch_tools import performance_tweaks
performance_tweaks(True, deterministic=None)

import torch
import visualbench as vb
from evolving_optimizers import *
from evolving_optimizers._tasks import optimizer
import numpy as np
import joblib
import copy
import torchzero as tz

torch.manual_seed(0)
torch.cuda.manual_seed(0)
random.seed(0)
np.random.seed(0)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE = "cpu"
bench = vb.Collinear(vb.models.MLP([32, 10]), batch_size=1).to(DEVICE).set_performance_mode(True).set_print_inverval()

In [ ]:
optimizer_operands = (optimizer.Grad, optimizer.GradAt, optimizer.DirectionalGrad, optimizer.Hessp, optimizer.HesspAt, optimizer.DirectionalHessp)

def evaluate_tree(tree: BaseOperation, print_progress=True):
    operands = tree.flat_branches()
    if len(operands) > 64:
        return float("inf")

    if not any(type(o) in optimizer_operands for o in operands):
        return float("inf")

    opt_fn = lambda p,lr: optimizer.TreeOpt(p, lr, tree)
    bench.tune(
        opt_fn, grid=[-4, -2, 0], metrics="test loss", log_scale=True, step=1,
        num_candidates=1, num_binary=3, num_expansions=3,
        max_passes=200, max_steps=100, print_progress=print_progress,
    )

    penalty = 0
    if len(tree.flat_branches()) > 32:
        penalty = (len(tree.flat_branches()) - 32) * 0.1

    test_loss = bench.logger.last("test loss")

    if not math.isfinite(test_loss):
        return float("inf")

    return float(test_loss + penalty)

bench._test_epoch()
print(f"{bench.logger.min('test loss') = }")
for opt in reversed(optimizer.init_population()):
    print(f"{evaluate_tree(opt, True)}")

In [ ]:
runner = Runner()
pool = RandomPool(COMMON_POOL + optimizer.OPTIMIZER_POOL)

opt = IslandGA([K1Plus1(), K1Plus1(), K1Plus1(), K1Plus1(), K1Plus1()], random_init=100)
# opt = K1Plus1()
# opt = ARS(0.9)

with DiskCachedObjective("evaluated.log", map_objective(evaluate_tree)) as objective:
    population = objective(optimizer.init_population())
    lowest_loss = float("inf")

    for i in range(1000):
        try:

            population = runner.step(objective, opt, population, pool)

            if runner.best_solutions[0].fitness < lowest_loss:
                print(f'{i}: {runner.best_solutions[0].fitness:.3f}, {runner.best_solutions[0].tree}')
                lowest_loss = runner.best_solutions[0].fitness
            else:
                print(f'{i}', end="\r")

        except KeyboardInterrupt:
            break

In [ ]:
for ind in population:
    print(ind.fitness, len(ind.tree.flat_branches()), ind.tree.string())

In [ ]:
for ind in runner.best_solutions:
    print(ind.fitness, len(ind.tree.flat_branches()), ind.tree.string())

In [ ]:
bench = vb.FunctionDescent("booth")#.set_noise(10,1)
opt = optimizer.TreeOpt(bench.parameters(), 1e-1, runner.best_solutions[0].tree)
bench.run(opt, 1000).plot()

In [ ]:
bench.render("logreg1")

In [ ]:
import joblib
joblib.dump(runner.best_solutions[0].tree, f"logreg1 {lowest_loss:.5f} K1Plus1.joblib")